# Ki-67 Model Validation in VS Code

This notebook validates the 3 trained Ki-67 models (InceptionV3, ResNet50, ViT) using the Ki67_Dataset_for_Colab dataset directly in VS Code.

## 📋 What we'll validate:
1. ✅ Load trained model weights (.pth files)
2. ✅ Test on Ki67_Dataset_for_Colab dataset  
3. ✅ Calculate comprehensive metrics
4. ✅ Create ensemble predictions
5. ✅ Generate confusion matrices
6. ✅ Compare results with training performance

In [ ]:
# Import required libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from pathlib import Path
import json
import pickle
import os
import zipfile
from datetime import datetime

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report
)

try:
    import timm
    print("✅ timm available for ViT model")
except ImportError:
    print("⚠️  timm not available, will use fallback CNN for ViT")
    timm = None

print("🔬 Ki-67 Model Validation System")
print("="*50)

# Setup device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

In [ ]:
# Setup paths (adjust these paths to match your local setup)
DATASET_ZIP_PATH = "/Users/chinthan/ki7/Ki67_Dataset_for_Colab.zip"  # Adjust this path
MODELS_PATH = "/Users/chinthan/ki7"  # Path where your .pth files are located
EXTRACT_PATH = "/Users/chinthan/ki7/ki67_validation_dataset"

def extract_dataset(dataset_zip_path, extract_path):
    """Extract the Ki67 dataset"""
    if os.path.exists(dataset_zip_path):
        print(f"✅ Found dataset at: {dataset_zip_path}")
        
        os.makedirs(extract_path, exist_ok=True)
        
        print("Extracting dataset...")
        with zipfile.ZipFile(dataset_zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
        
        print("✅ Dataset extracted successfully!")
        
        # List extracted contents
        for root, dirs, files in os.walk(extract_path):
            level = root.replace(extract_path, '').count(os.sep)
            indent = ' ' * 2 * level
            print(f'{indent}{os.path.basename(root)}/')
            subindent = ' ' * 2 * (level + 1)
            for file in files[:5]:  # Show first 5 files
                print(f'{subindent}{file}')
            if len(files) > 5:
                print(f'{subindent}... and {len(files)-5} more files')
        
        return extract_path
    else:
        print(f"❌ Dataset not found at: {dataset_zip_path}")
        print("Please update DATASET_ZIP_PATH to the correct location")
        return None

# Extract dataset
dataset_path = extract_dataset(DATASET_ZIP_PATH, EXTRACT_PATH)